In [1]:
#load up modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import pickle
import os

import category_encoders as ce

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm


import tensorflow as tf
from tensorflow import keras
from keras import layers

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Get dir
path="D:/Kuliah/Semester 5/PSD/psd-challenge/Kerja lah su/Challenge 2/"
os.listdir(path)

def savefile(result, savename):
    try:
        pd.DataFrame(zip(range(1,len(result)+1),result), columns=["No", "UHH"]).to_csv(path+savename+".csv", index=False)
    except:
        print("It only saves the result retard!")

In [3]:
#Get dataset                                #Gw baru nyadar train di split lagi?
test_data=pd.read_excel(path+"Data/Data Test.xlsx")
train=pd.read_excel(path+"Data/Data Train.xlsx")

In [4]:
#Get Metadatadata and each attribut encoder
Metadata=pd.read_excel(path+"Data/Metadata.xlsx")

# colnames w/ encoders
temp=[]
for i in Metadata.iloc[23:,0].unique():
    temp.append(i) if str(i).startswith("PSTV") else 0
temp

valueencoder={}
try:
    for i in range(len(temp)):
        temp1=Metadata.iloc[Metadata.iloc[23:,:][Metadata["Variable Information"]==temp[i]].index[0]:Metadata.iloc[23:,:][Metadata["Variable Information"]==temp[i+1]].index[0],:].values
        valueencoder[str(temp[i])] = (dict(zip(temp1[:,1], temp1[:,2])))
except:
    temp1=Metadata.iloc[Metadata.iloc[23:,:][Metadata["Variable Information"]=="PSTV17"].index[0]:,:].values
    valueencoder[str(temp[i])] = (dict(zip(temp1[:,1], temp1[:,2])))

In [5]:
X=train.iloc[:,:-1]
y=train.iloc[:,-1]

In [6]:
#convert to onehots
ordinalcolumns=[]
for i in list(valueencoder.keys()):
    ordinalcolumns.append(i) if i in list(train.columns)  else 0
ordinalcolumns

onehotencoder=ce.OneHotEncoder(cols=ordinalcolumns)
onehotencoder.fit(X)
X=onehotencoder.transform(X)
test_data=onehotencoder.transform(test_data)

In [7]:
def r2score(y_true, y_pred):
    SS_res =  keras.backend.sum(keras.backend.square( y_true-y_pred ))
    SS_tot = keras.backend.sum(keras.backend.square(y_true - keras.backend.mean(y_true)))
    return (1 - (SS_res/SS_tot))

In [8]:
def build_and_compile_model():
  model = keras.Sequential([
      layers.Dense(350, input_dim=X.shape[1], activation='relu'),
      layers.Dense(350, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_percentage_error',
                optimizer=tf.keras.optimizers.Adam(), metrics=[r2score, "mean_absolute_percentage_error"])
  return model

In [9]:
train_data, validation_data, train_value, validation_value= train_test_split(X, y, train_size=0.9)

dnn_model = build_and_compile_model()

history=dnn_model.fit(X, y, verbose=1, validation_data=(validation_data, validation_value), epochs=30, batch_size=64)

Epoch 1/30
143/143 [==============================] - 3s 7ms/step - loss: 93.2050 - r2score: -129.5060 - mean_absolute_percentage_error: 93.2050 - val_loss: 40.0902 - val_r2score: -8.1341 - val_mean_absolute_percentage_error: 40.0902
Epoch 2/30
143/143 [==============================] - 1s 4ms/step - loss: 71.1527 - r2score: -40.0801 - mean_absolute_percentage_error: 71.1527 - val_loss: 45.8272 - val_r2score: -10.5426 - val_mean_absolute_percentage_error: 45.8272
Epoch 3/30
143/143 [==============================] - 1s 4ms/step - loss: 41.5537 - r2score: -9.9545 - mean_absolute_percentage_error: 41.5537 - val_loss: 56.3854 - val_r2score: -14.5019 - val_mean_absolute_percentage_error: 56.3854
Epoch 4/30
143/143 [==============================] - 1s 4ms/step - loss: 45.6862 - r2score: -13.7317 - mean_absolute_percentage_error: 45.6862 - val_loss: 51.8470 - val_r2score: -12.1297 - val_mean_absolute_percentage_error: 51.8470
Epoch 5/30
143/143 [==============================] - 1s 5ms/step